# Advent of Code 2021 - Day 7

In [7]:
typealias Segments = List<String>
data class Input(val signalPatterns: Segments, val outputs: Segments)

In [8]:
import java.io.File
import java.util.Scanner

val regex = Regex("""(\w+) (\w+) (\w+) (\w+) (\w+) (\w+) (\w+) (\w+) (\w+) (\w+) \| (\w+) (\w+) (\w+) (\w+)""")

val inputs: List<Input> = File("Day8.input.txt")
    .bufferedReader()
    .readLines()
    .map { regex.find(it)!!.destructured.toList() }
    .map { Input(signalPatterns = it.slice(0..9), outputs = it.slice(10..13)) }

## Part 1

Given an `Input`, each `signalPatterns` and `outputs` `Segments` represents a digit in a seven-segment display. For instance to render `1` a segment could contain `c` and `f`. The `signalPatterns` for a given `Input` contain the `Segments` for all nine digits, however, the specific segments are mixed up from input to input. `outputs` contains four digits derived from the signal patterns. In all the `outputs` how many times do digits `1`, `4`, `7`, or `8` appear? 

In [9]:
inputs.sumOf { it.outputs.count { it.length == 7 || it.length == 4 || it.length == 3 || it.length == 2 } }

476

### Notes

This is pretty trivial, but it primes us to think about the characteristics of a seven-segment display. The key here is that, digits 1, 4, 7, and 8 all only use a specific number of segments (2, 3, 4, 7 respectively) so we can just check the lengths to determine what digit they belong to.

## Part 2

Decode each `Input`'s `signalPatterns` given that all ten digits are encoded, e.g. figure out which string belongs to which digit. Use the decoded values to match each `outputs`' segments to a digit. Sum all the decoded outputs.

In [10]:
fun Segments.filterLength(length: Int) = filter { it.length == length }

fun Segments.findOne(): String = first { it.length == 2 }
fun Segments.findFour(): String = first { it.length == 4 }
fun Segments.findSeven(): String = first { it.length == 3 }
fun Segments.findEight(): String = first { it.length == 7 }

fun Segments.findZero(): String = (findFour() to findSeven()).let { (four, seven) ->
    filterLength(6).first { pattern -> !four.all { it in pattern } && seven.all { it in pattern } }
}

fun Segments.findSix(): String = findSeven().let { seven ->
    filterLength(6).first { pattern -> !seven.all { it in pattern } }
}

fun Segments.findNine(): String = findFour().let { four ->
    filterLength(6).first { pattern -> four.all { it in pattern } }
}

fun Segments.findTwo(): String = (findFour() to findEight())
    .let { (four, eight) -> eight.filterNot { it in four } }
    .let { diff ->
        filterLength(5).first { pattern -> diff.all { it in pattern } }
    }

fun Segments.findThree(): String = (findOne()).let { one -> 
    filterLength(5).first { pattern -> one.all { it in pattern } }
}

fun Segments.findFive(): String = (findOne() to findFour())
    .let { (one, four) -> four.filterNot { it in one } }
    .let { diff ->
        filterLength(5).first { pattern -> diff.all { it in pattern } }
    }

fun Segments.findNumbers(): Segments = listOf(findZero(),findOne(),findTwo(),
    findThree(),findFour(),findFive(),findSix(),findSeven(),findEight(),findNine())

fun Segments.decode(outputs: Segments): Int {
    val numbers = findNumbers()

    return outputs.joinToString("") { output ->
        numbers.indexOfFirst { number -> output.length == number.length && output.all { it in number } }.toString()
    }.toInt()
}

inputs.sumOf { it.signalPatterns.decode(it.outputs) }

1011823

### Notes

This one was pretty tough for me and I only started making progress when I started to physically draw the problem out. 

The key here is to notice the patterns when you "add" or "subtract" digits (physically, not numerically). For instance, if you combine the segments for `4` and `7` it actually makes the segment `9`. In other words, segments `4` and `7` overlap to make `9`. Since we know `4` and `7` automatically, we can find `9` by just checking if it contains all of the segments from `4` and `7`.

Unfortunately, I could not figure out how to "compose" digits to create all of the numbers, or find uniqueness among all of the numbers. The next key point for me to notice that rather than finding out, for instance, what makes `6` unique among all digits, you only need to find out what makes `6` unique vs `0` and `9` - because those are the only other digits that are the same length. This is similar for `2`, `3`, and `5`. The rules I found are as follows:

- `0`: Does not contain `4`, contains `7`.
- `6`: Does not contain `7`.
- `9`: Contains `4`.
- `2`: Contains _the difference_ of `4` and `8`.
- `3`: Contains `1`.
- `5`: Contains _the difference_ of `4` and `1`.

The code could be better, but I just want to move on from this problem as soon as possible.